In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, round

# Create Spark session
spark = SparkSession.builder.appName("CostOptimizedPipeline").getOrCreate()

# Generate sample data
data = [(i, f"Product_{i%10}", i%5, 2024) for i in range(1, 100001)]
columns = ["order_id", "product", "category_id", "year"]

df = spark.createDataFrame(data, columns)

# Add 'amount' column with random values
df = df.withColumn("amount", round(rand(), 2))

df.show(5)

+--------+---------+-----------+----+------+
|order_id|  product|category_id|year|amount|
+--------+---------+-----------+----+------+
|       1|Product_1|          1|2024|   0.0|
|       2|Product_2|          2|2024|  0.28|
|       3|Product_3|          3|2024|  0.05|
|       4|Product_4|          4|2024|  0.23|
|       5|Product_5|          0|2024|  0.84|
+--------+---------+-----------+----+------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import col

# Filter orders with amount > 0.1 and select required columns
df_transformed = df.filter(col("amount") > 0.1) \
                   .select("order_id", "product", "category_id", "year", "amount")

df_transformed.show(5)

+--------+---------+-----------+----+------+
|order_id|  product|category_id|year|amount|
+--------+---------+-----------+----+------+
|       2|Product_2|          2|2024|  0.28|
|       4|Product_4|          4|2024|  0.23|
|       5|Product_5|          0|2024|  0.84|
|       6|Product_6|          1|2024|  0.44|
|       7|Product_7|          2|2024|  0.66|
+--------+---------+-----------+----+------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import sum, avg, round

# Compute total and average amount per category
df_summary = df_transformed.groupBy("category_id") \
                           .agg(
                               round(sum("amount"),2).alias("total_amount"),
                               round(avg("amount"),2).alias("avg_amount")
                           )

display(df_summary)

category_id,total_amount,avg_amount
0,9909.22,0.55
2,9900.6,0.55
1,9871.78,0.55
3,9875.75,0.55
4,9887.28,0.55


In [0]:
# Install openpyxl in the cluster
%pip install openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Option 1
%restart_python

In [0]:
df_full_pd = df_transformed.toPandas()
df_summary_pd = df_summary.toPandas()

In [0]:
df_full_pd.to_excel("cost_optimized_full_data.xlsx", index=False)
df_summary_pd.to_excel("cost_optimized_summary.xlsx", index=False)

print("Full dataset and summary saved successfully!")

Full dataset and summary saved successfully!


In [0]:
# Show first 100 rows in Databricks notebook
display(df_transformed.limit(100))

order_id,product,category_id,year,amount
2,Product_2,2,2024,0.28
4,Product_4,4,2024,0.23
5,Product_5,0,2024,0.84
6,Product_6,1,2024,0.44
7,Product_7,2,2024,0.66
8,Product_8,3,2024,0.8
9,Product_9,4,2024,0.47
10,Product_0,0,2024,0.33
11,Product_1,1,2024,0.36
12,Product_2,2,2024,0.15


In [0]:
spark.stop()